In [1]:
import pandas as pd
import numpy as np

In [2]:
!wget https://github.com/Moradnejad/ColBERT-Using-BERT-Sentence-Embedding-for-Humor-Detection/raw/master/Data/dataset.csv
!wget https://github.com/Moradnejad/ColBERT-Using-BERT-Sentence-Embedding-for-Humor-Detection/raw/master/Data/test.csv
!wget https://github.com/Moradnejad/ColBERT-Using-BERT-Sentence-Embedding-for-Humor-Detection/raw/master/Data/train.csv
!wget https://github.com/Moradnejad/ColBERT-Using-BERT-Sentence-Embedding-for-Humor-Detection/raw/master/Data/dev.csv

--2024-06-02 19:21:58--  https://github.com/Moradnejad/ColBERT-Using-BERT-Sentence-Embedding-for-Humor-Detection/raw/master/Data/dataset.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Moradnejad/ColBERT-Using-BERT-Sentence-Embedding-for-Humor-Detection/master/Data/dataset.csv [following]
--2024-06-02 19:21:59--  https://raw.githubusercontent.com/Moradnejad/ColBERT-Using-BERT-Sentence-Embedding-for-Humor-Detection/master/Data/dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14874240 (14M) [text/plain]
Saving to: ‘dataset.csv’

dataset.csv         100%[===================>]  14.18M

In [3]:
import pandas as pd

# prendo dal train per scrupolo
df_train = pd.read_csv("train.csv")

# Sostituisci i valori true/false in "humor" con "funny" e "not funny" come nel loro excel
df_train['humor'] = df_train['humor'].replace({True: 'funny', False: 'not funny'})


print(df_train.head(10))

                                                text      humor
0  Joe biden rules out 2020 bid: 'guys, i'm not r...  not funny
1  Watch: darvish gave hitter whiplash with slow ...  not funny
2  What do you call a turtle without its shell? d...      funny
3      5 reasons the 2016 election feels so personal  not funny
4  Pasco police shot mexican migrant from behind,...  not funny
5  Martha stewart tweets hideous food photo, twit...  not funny
6  What is a pokemon master's favorite kind of pa...      funny
7  Why do native americans hate it when it rains ...      funny
8  Obama's climate change legacy is impressive, i...  not funny
9      My family tree is a cactus, we're all pricks.      funny


In [4]:
import random

# 5 frasi random not funny
random_sentences_0 = df_train[df_train['humor'] == "not funny"]['text'].sample(10).reset_index(drop=True)

# 5 frasi random funny
random_sentences_1 = df_train[df_train['humor'] == "funny"]['text'].sample(10).reset_index(drop=True)


sentences_group_0 = list(random_sentences_0)
sentences_group_1 = list(random_sentences_1)

In [5]:
print(sentences_group_1)

['What do you call a fat taylor swift? taylor not so swift', "Why shouldn't you post to reddit while driving? becau", 'Just landed my dream job of before picture', "What is an italian with alzheimer's favorite espresso drink? affogato!", "I came into some money yesterday. i couldn't find the tissues.", 'If people say that love finds you... then i must be waldo.', 'I still not comfortable with how we spell coffee.', "I'm positive i lost an electron... ...better keep an ion that.", 'Sorry i asked for a rim job... ... it was only tongue in cheek.', 'Found a copy of dr. no in german, and it only set me back nein euro.']


In [7]:
## test (preso dall'excel per avere stesse frasi causa random.seed)

import pandas as pd

df_test = pd.read_csv("promptest.tsv", sep='\t')

print(df_test.head(10))

                                              prompt  \
0  Which one of the attributes: "funny", "not fun...   
1  Which one of the attributes: "funny", "not fun...   
2  Which one of the attributes: "funny", "not fun...   
3  Which one of the attributes: "funny", "not fun...   
4  Which one of the attributes: "funny", "not fun...   
5  Which one of the attributes: "funny", "not fun...   
6  Which one of the attributes: "funny", "not fun...   
7  Which one of the attributes: "funny", "not fun...   
8  Which one of the attributes: "funny", "not fun...   
9  Which one of the attributes: "funny", "not fun...   

                                                text annotation  
0  Black teen's response to violence in his commu...  not funny  
1  King splits his kingdom to his three sons... h...      funny  
2  If you're between a rock and a hard place ... ...      funny  
3  What did a black twin called his brother befor...      funny  
4  For women, perky breasts will one day be a dis... 

In [8]:
# nuove colonne per i prompt
for n in [10, 8, 6, 4, 2]:
    df_test[f'prompt{n}'] = (
        'Which one of the attributes: "funny", "not funny" describes a given text?\n\n'
        "Write your answer in the form of a Python list containing the appropriate attribute.\n"
        'These are examples of "funny" texts:\n' +
        "\n".join(f"- {sentence}" for sentence in random_sentences_1[:n]) +
        '\n\nThese are examples of "not funny" texts:\n' +
        "\n".join(f"- {sentence}" for sentence in random_sentences_0[:n]) +
        "\n\nText: " + df_test['text']
    )

print(df_test.head(10))

                                              prompt  \
0  Which one of the attributes: "funny", "not fun...   
1  Which one of the attributes: "funny", "not fun...   
2  Which one of the attributes: "funny", "not fun...   
3  Which one of the attributes: "funny", "not fun...   
4  Which one of the attributes: "funny", "not fun...   
5  Which one of the attributes: "funny", "not fun...   
6  Which one of the attributes: "funny", "not fun...   
7  Which one of the attributes: "funny", "not fun...   
8  Which one of the attributes: "funny", "not fun...   
9  Which one of the attributes: "funny", "not fun...   

                                                text annotation  \
0  Black teen's response to violence in his commu...  not funny   
1  King splits his kingdom to his three sons... h...      funny   
2  If you're between a rock and a hard place ... ...      funny   
3  What did a black twin called his brother befor...      funny   
4  For women, perky breasts will one day be a di

In [9]:
# test su csv
df_test.head(10).to_csv("output_file.csv", index=False)

In [10]:
# colonne per csv totale
selected_columns = ["text", "annotation", "prompt10", "prompt8", "prompt6", "prompt4", "prompt2"]

# Salva il DataFrame con solo le colonne selezionate in un file CSV
df_test[selected_columns].to_csv("04_ColBERT.csv", index=False)